In [1]:
%matplotlib inline
from solvers import solver
import numpy as np
import qtools as qt
import tucker3d as tuck

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 29 days
openbabel not found in path, switching to PyQuante backend
libint extension not found, switching to normal ERI computation


In [ ]:
name = 'H2'
molecule = qt.molecule(name)

gradients = []
for n in [1024, 2048, 4096, 8192, 2*8192]:
    sol = solver(molecule, method='lda', mixing=2, eps=1e-7,
                 maxiter=60, meshsize=n, boxsize=10.)
    sol.solve()
    gradients.append(gradient_diatomic(sol))
    print gradients

In [48]:
a, b, c = gradient_diatomic(sol)
print a, b, c

0.0144457588898 -0.247879161371 0.510204081633


In [5]:
2*a - b

(-0.0077936754052120305+0j)

In [8]:
2*a - b

(-0.0040150819633524293+0j)

In [11]:
2*a - b

(-0.0060791142769833773+0j)

In [14]:
2*a - b

(0.00070546547821748185+0j)

In [4]:
2*a - b

(-0.0060777468614038321+0j)

In [16]:
x = sol.params.grid

In [17]:
tuck.dot(sol.rho, tuck.ones(sol.rho.n))*(x[1]-x[0])**3

0.99999476156725564

In [24]:
1./0.52917721092 * 2 * (0.7/(1./0.52917721092))

1.4

In [15]:
1./(1.4)**2

0.5102040816326532

In [42]:
def gradient_diatomic(self):

        # pot squared
        #tensor_x = charge*tuck.round(vec[0]*tuck.ones((N, N, N)) - tensor_x, self.params.eps)
        #qt.pots.coulomb(x, vec, self.params.ind, self.params.eps, beta=3.0)
        #d_coulomb -= tuck.cross.multifun([tensor_x, pot_squared], self.params.eps, lambda x: x[0]*x[1])

        molecule = self.molecule
        x = self.params.grid
        h = x[1] - x[0]
        N = len(self.params.grid)
        Norb = self.molecule.orbitals
        num_atoms = self.molecule.num_atoms

        i = 0
        vec = molecule.atoms[i].rad
        charge = molecule.atoms[i].charge
        def fun_coulomb_squared((i, j, k)):
            return charge*(x[i] - vec[0]) / np.sqrt((vec[0] - x[i])**2 + (vec[1] - x[j])**2 + (vec[2] - x[k])**2)**(3.0)
        d_coulomb = tuck.cross.cross3d(fun_coulomb_squared, N, self.params.eps)
        d_coulomb = tuck.round(tuck.real(d_coulomb), self.params.eps)

        self.get_rho()
        #d_coulomb = tuck.cross.multifun([self.rho, d_coulomb], self.params.eps, lambda x: x[0]*x[1])
        
        
        dE_nuc = 0.0
        j = 1
        i = 0
        vec_i = molecule.atoms[i].rad
        charge_i = molecule.atoms[i].charge
        vec_j = molecule.atoms[j].rad
        charge_j = molecule.atoms[j].charge
        dE_nuc = (dE_nuc + (vec_i[0] - vec_j[0])*charge_i*charge_j/
                  np.sqrt((vec_i[0] - vec_j[0])**2 + (vec_i[1] - vec_j[1])**2 + (vec_i[2] - vec_j[2])**2)**3)

        return tuck.dot(d_coulomb, 2*self.rho)*(x[1]-x[0])**3 + dE_nuc

In [35]:
gradient_diatomic(sol)

((-0.007117600384885292+0j), (-0.25866084100876924+0j), 0.51020408163265318)

In [11]:
gradient_diatomic(sol)

((-0.25814740224407762+0j), -0.51020408163265318)

In [27]:
2*0.2581474047257-0.51020408163

0.006090727821400077

In [30]:
0.01646647 / 0.00609072782

2.703530758003893